In [ ]:
import pandas as pd
import fastf1

fastf1.Cache.enable_cache("../../data/cache")

original_df = pd.read_csv('../../data/processed/export_v1.csv')

races_df = pd.read_csv('../../data/raw_data/races.csv')

original_df.head()


In [ ]:
df = original_df[['raceId']]
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df = pd.merge(df, races_df, on='raceId', how='left')

df = df[df['year'] >= 2018]
df.head()


In [ ]:
races = []
for idx, row in df.iterrows():
    try:
        fp1 = fastf1.get_session(year=row['year'], gp=row['name'], identifier='fp1')
    except:
        fp1 = None
    try:
        fp2 = fastf1.get_session(year=row['year'], gp=row['name'], identifier='fp2')
    except:
        fp2 = None
    try:
        fp3 = fastf1.get_session(year=row['year'], gp=row['name'], identifier='fp3')
    except:
        fp3 = None
    try:
        quali = fastf1.get_session(year=row['year'], gp=row['name'], identifier='q')
    except:
        quali = None

    races.append({
        'raceId': row['raceId'],
        'fp1_session': fp1,
        'fp2_session': fp2,
        'fp3_session': fp3,
        'qualifying': quali
    })

In [ ]:
race = races[19]['fp1_session']
race.load(laps=True, weather=False, telemetry=False, messages=False)

In [ ]:
laps = race.laps
laps = laps[laps['Deleted'] == False]
quick_laps = laps.pick_quicklaps()

quick_laps.head()

In [ ]:
from src.util import convert_timedelta_columns_to_millis

quick_laps = quick_laps.copy()
convert_timedelta_columns_to_millis(quick_laps, ['Sector1Time', 'Sector2Time', 'LapTime'], in_place=True)

quick_laps


In [ ]:
maximum = laps.groupby("Driver").max(numeric_only=True)

laps_driven = maximum['LapNumber']

maximum

In [ ]:

grp = quick_laps.groupby("Driver").mean(numeric_only=True)

fp1_df = pd.DataFrame({
    'driver': grp.index,
    'fp1_avg_sector_1': grp['Sector1Time'],
    'fp1_avg_sector_2': grp['Sector2Time'],
    'fp1_avg_lap_time': grp['LapTime'],
    'fp1_avg_speedI1': grp['SpeedI1'],
    'fp1_avg_speedI2': grp['SpeedI2'],
    'fp1_avg_speedFL': grp['SpeedFL'],
    'fp1_avg_speedST': grp['SpeedST'],
    'fp1_avg_tyre_life': grp['TyreLife'],
    'fp1_avg_is_on_fresh_tyres': grp['FreshTyre']
})
grp

In [ ]:
session = races[1]['qualifying']
session.load()

In [ ]:
results = session.results
results.head()

In [ ]:


convert_timedelta_columns_to_millis(results, ['Q1', 'Q2', 'Q3'], in_place=True)

qualifying_positions = results['Position']
q1_time = results['Q1']
q2_time = results['Q2']
q3_time = results['Q3']

In [ ]:
qualifying_df = pd.DataFrame({
    'raceId': races[19]['raceId'],
    'driver': results['Abbreviation'],
    'q1_time': q1_time,
    'q2_time': q2_time,
    'q3_time': q3_time,
    'q_position': qualifying_positions
})

In [ ]:
session_df = pd.merge(qualifying_df, fp1_df, on='driver', how='left')

session_df

In [ ]:
full_df = pd.merge(original_df, session_df, on='raceId', how='left')

full_df